# Notebook 03: Construcción de **analytics.obt_trips** (solo Snowflake)

**Objetivo:** Ensamblar la OBT con columnas derivadas y metadatos a partir de `analytics.trips_enriched_unified.

### Checklist
- Grano: **1 fila = 1 viaje**
- Derivadas: `trip_duration_min`, `avg_speed_mph`, `tip_pct`
- Calendario: `pickup_date`, `pickup_hour`, `dropoff_date`, `dropoff_hour`, `day_of_week_iso`, `month`, `year`
- Lineage: `run_id`, `source_year`, `source_month`, `service_type`
- Idempotencia: `CREATE OR REPLACE TABLE analytics.obt_trips`


## 1) Parámetros y conexión

In [29]:
import os
import snowflake.connector as sf

SF_ACCOUNT   = os.getenv('SNOWFLAKE_ACCOUNT')
SF_USER      = os.getenv('SNOWFLAKE_USER')
SF_PASSWORD  = os.getenv('SNOWFLAKE_PASSWORD')
SF_ROLE      = os.getenv('SNOWFLAKE_ROLE', 'SYSADMIN')
SF_WAREHOUSE = os.getenv('SNOWFLAKE_WAREHOUSE')
SF_DATABASE  = os.getenv('SNOWFLAKE_DATABASE')
SCHEMA_RAW   = os.getenv('SNOWFLAKE_SCHEMA_RAW', 'RAW')
SCHEMA_AN    =  os.getenv('SNOWFLAKE_SCHEMA_ANALYTICS','analytics')

ENRICHED_TABLE = os.getenv('AN_ENRICHED_TABLE', 'TRIPS_ENRICHED_UNIFIED')
OBT_TABLE      = os.getenv('AN_OBT_TABLE', 'OBT_TRIPS')

assert SF_ACCOUNT and SF_USER and SF_PASSWORD and SF_WAREHOUSE and SF_DATABASE, 'Faltan variables de conexión a Snowflake'
print('DB:', SF_DATABASE, '| Schemas:', SCHEMA_RAW, '→', SCHEMA_AN)
print('Input:', f'{SCHEMA_AN}.{ENRICHED_TABLE}', '| Output:', f'{SCHEMA_AN}.{OBT_TABLE}')

DB: SPARK_DATA | Schemas: SPARK_DATA.RAW → SPARK_DATA.analytics
Input: SPARK_DATA.analytics.TRIPS_ENRICHED_UNIFIED | Output: SPARK_DATA.analytics.OBT_TRIPS


### Helper de conexión/SQL

In [2]:
def snowflake_conn(schema: str | None = None):
    ctx = sf.connect(
        account=SF_ACCOUNT,
        user=SF_USER,
        password=SF_PASSWORD,
        warehouse=SF_WAREHOUSE,
        role=SF_ROLE,
        database=SF_DATABASE,
        schema=schema or SCHEMA_AN,
        client_session_keep_alive=True,
    )
    c = ctx.cursor()
    try:
        c.execute(f"USE DATABASE {SF_DATABASE}")
        c.execute(f"USE SCHEMA {schema or SCHEMA_AN}")
    finally:
        c.close()
    return ctx

def run_sql(sql: str, schema: str | None = None):
    with snowflake_conn(schema or SCHEMA_AN) as conn:
        cur = conn.cursor()
        try:
            cur.execute(f"USE DATABASE {SF_DATABASE}")
            cur.execute(f"USE SCHEMA {schema or SCHEMA_AN}")
            res = cur.execute(sql)
            try:
                return res.fetchall()
            except Exception:
                return []
        finally:
            cur.close()
print('✓ Helpers listos')

✓ Helpers listos


## 2) Crear la OBT con derivadas y metadatos

In [32]:
# === 3A: Crear OBT (estructura completa exigida por el PDF) ===
sql_create_obt = f'''
CREATE TABLE IF NOT EXISTS {SCHEMA_AN}.{OBT_TABLE} (
  -- Tiempo crudo y derivadas
  pickup_datetime       TIMESTAMP_NTZ,
  dropoff_datetime      TIMESTAMP_NTZ,
  pickup_date           DATE,
  pickup_hour           NUMBER(2,0),
  dropoff_date          DATE,
  dropoff_hour          NUMBER(2,0),
  day_of_week           STRING,   
  month                 NUMBER(2,0),
  year                  NUMBER(4,0),

  -- Ubicación
  pu_location_id        NUMBER,
  pu_zone               STRING,
  pu_borough            STRING,
  do_location_id        NUMBER,
  do_zone               STRING,
  do_borough            STRING,
  pu_service_zone        STRING,   
  do_service_zone        STRING,   

  -- Servicio y códigos
  service_type          STRING,   -- yellow/green
  vendor_id             NUMBER,
  vendor_name           STRING,
  rate_code_id          NUMBER,
  rate_code_desc        STRING,
  payment_type          NUMBER,
  payment_type_desc     STRING,
  trip_type             NUMBER,    -- green
  trip_type_desc        STRING,

  -- Viaje
  passenger_count       FLOAT,
  trip_distance         FLOAT,
  store_and_fwd_flag    STRING,

  -- Tarifas (mínimas del PDF)
  fare_amount           FLOAT,
  extra                 FLOAT,
  mta_tax               FLOAT,
  tip_amount            FLOAT,
  tolls_amount          FLOAT,
  improvement_surcharge FLOAT,
  congestion_surcharge  FLOAT,
  airport_fee           FLOAT,     -- 
  total_amount          FLOAT,
  e_hail_fee            FLOAT,     -- 

  -- Derivadas obligatorias
  trip_duration_min     FLOAT,
  avg_speed_mph         FLOAT,
  tip_pct               FLOAT,

  -- Lineage/Calidad (mínimos del PDF)
  run_id                STRING,
  ingested_at_utc       TIMESTAMP_NTZ,  
  source_service        STRING,         
  source_year           INT,
  source_month          INT,
  source_path           STRING,

  -- Metadata NB03 e idempotencia
  nb03_run_id           STRING,
  nb03_processed_at_utc TIMESTAMP_NTZ,
  trip_hash             STRING
)
CLUSTER BY (pickup_date, service_type, pu_location_id, do_location_id);
'''
run_sql(sql_create_obt, schema=SCHEMA_AN)
print('✓ OBT definida/creada:', f'{SCHEMA_AN}.{OBT_TABLE}')


✓ OBT definida/creada: SPARK_DATA.analytics.OBT_TRIPS


In [12]:
# ==== Detectar columnas en {SCHEMA_AN}.{ENRICHED_TABLE} y construir expresiones seguras ====
def split_db_schema(schema_str: str):
    # Acepta "DB.SCHEMA" o solo "SCHEMA" (usamos SF_DATABASE de fallback)
    parts = schema_str.split('.', 1)
    if len(parts) == 2:
        return parts[0], parts[1]
    return SF_DATABASE, schema_str

DB_AN, SCHEMA_ONLY_AN = split_db_schema(SCHEMA_AN)

def exists_col(db:str, schema:str, table:str, column:str) -> bool:
    q = f"""
    SELECT COUNT(*) 
    FROM {db}.INFORMATION_SCHEMA.COLUMNS
    WHERE TABLE_SCHEMA = %s
      AND TABLE_NAME   = %s
      AND UPPER(COLUMN_NAME) = UPPER(%s)
    """
    with snowflake_conn(f"{db}.{schema}") as conn:
        cur = conn.cursor()
        try:
            cur.execute(q, (SCHEMA_ONLY_AN, ENRICHED_TABLE, column))
            return cur.fetchone()[0] > 0
        finally:
            cur.close()

#Verificaciones para las columnas que aparecen en una tabla , pero no en la otra
has_src_service_type = exists_col(DB_AN, SCHEMA_ONLY_AN, ENRICHED_TABLE, "SRC_SERVICE_TYPE")
has_trip_type_desc   = exists_col(DB_AN, SCHEMA_ONLY_AN, ENRICHED_TABLE, "TRIP_TYPE_DESC")
has_pu_service_zone  = exists_col(DB_AN, SCHEMA_ONLY_AN, ENRICHED_TABLE, "PU_SERVICE_ZONE")
has_do_service_zone  = exists_col(DB_AN, SCHEMA_ONLY_AN, ENRICHED_TABLE, "DO_SERVICE_ZONE")
has_airport_fee      = exists_col(DB_AN, SCHEMA_ONLY_AN, ENRICHED_TABLE, "AIRPORT_FEE")
has_e_hail_fee       = exists_col(DB_AN, SCHEMA_ONLY_AN, ENRICHED_TABLE, "E_HAIL_FEE")

print(
    "ENRICHED cols →",
    {"src_service_type": has_src_service_type,
     "trip_type_desc": has_trip_type_desc,
     "pu_service_zone": has_pu_service_zone,
     "do_service_zone": has_do_service_zone,
     "airport_fee": has_airport_fee,
     "e_hail_fee": has_e_hail_fee}
)

# Expresiones seguras (solo referenciamos columnas si existen)
SRC_SERVICE_EXPR   = "e.src_service_type" if has_src_service_type else "e.service_type"
TRIP_TYPE_DESC_EXPR= "e.trip_type_desc"   if has_trip_type_desc   else "CASE e.trip_type WHEN 1 THEN 'Street-hail' WHEN 2 THEN 'Dispatch' END"
PU_SVC_ZONE_EXPR   = "e.pu_service_zone"  if has_pu_service_zone  else "NULL::STRING"
DO_SVC_ZONE_EXPR   = "e.do_service_zone"  if has_do_service_zone  else "NULL::STRING"
AIRPORT_FEE_EXPR   = "e.airport_fee"      if has_airport_fee      else "NULL::FLOAT"
E_HAIL_FEE_EXPR    = "e.e_hail_fee"       if has_e_hail_fee       else "NULL::FLOAT"

# ==== STG OBT ====
sql_stg_obt = f"""
CREATE OR REPLACE TABLE {SCHEMA_AN}.{OBT_TABLE}_STG AS
WITH base AS (
  SELECT
    e.pickup_datetime::TIMESTAMP_NTZ   AS pickup_datetime,
    e.dropoff_datetime::TIMESTAMP_NTZ  AS dropoff_datetime,

    -- Derivadas de tiempo
    TO_DATE(e.pickup_datetime)                         AS pickup_date,
    EXTRACT(HOUR FROM e.pickup_datetime)               AS pickup_hour,
    TO_DATE(e.dropoff_datetime)                        AS dropoff_date,
    EXTRACT(HOUR FROM e.dropoff_datetime)              AS dropoff_hour,
    TO_CHAR(e.pickup_datetime, 'DY')                   AS day_of_week,
    EXTRACT(MONTH FROM e.pickup_datetime)              AS month,
    EXTRACT(YEAR  FROM e.pickup_datetime)              AS year,

    -- Ubicación (con zones opcionales)
    e.pulocationid                                     AS pu_location_id,
    e.pu_zone, e.pu_borough,
    e.dolocationid                                     AS do_location_id,
    e.do_zone, e.do_borough,
    {PU_SVC_ZONE_EXPR}                                 AS pu_service_zone,
    {DO_SVC_ZONE_EXPR}                                 AS do_service_zone,

    -- Servicio y catálogos
    e.service_type,
    e.vendorid                                         AS vendor_id,
    e.vendor_name,
    e.ratecodeid                                       AS rate_code_id,
    e.rate_code_desc,
    e.payment_type,
    e.payment_type_desc,
    e.trip_type,
    {TRIP_TYPE_DESC_EXPR}                              AS trip_type_desc,

    -- Viaje
    e.passenger_count, e.trip_distance, e.store_and_fwd_flag,

    -- Tarifas
    e.fare_amount, e.extra, e.mta_tax, e.tip_amount, e.tolls_amount,
    e.improvement_surcharge, e.congestion_surcharge,
    {AIRPORT_FEE_EXPR}                                 AS airport_fee,
    e.total_amount,
    {E_HAIL_FEE_EXPR}                                  AS e_hail_fee,

    -- Lineage NB02 (source_service con fallback seguro)
    e.run_id, e.ingested_at_utc, e.source_year, e.source_month,
    {SRC_SERVICE_EXPR}                                  AS source_service,
    e.source_path,

    -- Derivadas crudas
    DATEDIFF('minute', e.pickup_datetime, e.dropoff_datetime) AS trip_duration_min_raw

  FROM {SCHEMA_AN}.{ENRICHED_TABLE} e
  WHERE
    e.pickup_datetime IS NOT NULL
    AND e.dropoff_datetime IS NOT NULL
    AND e.dropoff_datetime >= e.pickup_datetime
    AND e.trip_distance >= 0
    AND e.total_amount IS NOT NULL
)
SELECT
  pickup_datetime, dropoff_datetime,
  pickup_date, pickup_hour, dropoff_date, dropoff_hour, day_of_week, month, year,

  pu_location_id, pu_zone, pu_borough, do_location_id, do_zone, do_borough,
  pu_service_zone, do_service_zone,

  service_type, vendor_id, vendor_name, rate_code_id, rate_code_desc,
  payment_type, payment_type_desc, trip_type, trip_type_desc,

  passenger_count, trip_distance, store_and_fwd_flag,

  fare_amount, extra, mta_tax, tip_amount, tolls_amount,
  improvement_surcharge, congestion_surcharge, airport_fee, total_amount, e_hail_fee,

  -- Derivadas con defensas
  trip_duration_min_raw AS trip_duration_min,
  IFF(trip_distance > 0 AND trip_duration_min_raw > 0,
      trip_distance / (trip_duration_min_raw / 60.0), NULL)      AS avg_speed_mph,
  IFF(total_amount > 0, tip_amount / total_amount, NULL)         AS tip_pct,

  run_id, ingested_at_utc, source_service, source_year, source_month, source_path,

  '{os.getenv("RUN_ID", "NB03")}' AS nb03_run_id,
  CURRENT_TIMESTAMP()::TIMESTAMP_NTZ AS nb03_processed_at_utc,

  -- trip_hash compatible con NB02
  MD5(
    COALESCE(service_type,'') || '|' ||
    COALESCE(TO_VARCHAR(pickup_datetime,'YYYY-MM-DD HH24:MI:SS.FF3'),'') || '|' ||
    COALESCE(TO_VARCHAR(dropoff_datetime,'YYYY-MM-DD HH24:MI:SS.FF3'),'') || '|' ||
    COALESCE(vendor_id,0) || '|' ||
    COALESCE(pu_location_id,0) || '|' ||
    COALESCE(do_location_id,0) || '|' ||
    COALESCE(ROUND(total_amount,2),0)
  ) AS trip_hash
FROM base;
"""
run_sql(sql_stg_obt, schema=SCHEMA_AN)
print('✓ STG OBT listo:', f'{SCHEMA_AN}.{OBT_TABLE}_STG')


ENRICHED cols → {'src_service_type': False, 'trip_type_desc': False, 'pu_service_zone': False, 'do_service_zone': False, 'airport_fee': False, 'e_hail_fee': False}
✓ STG OBT listo: SPARK_DATA.analytics.OBT_TRIPS_STG


In [38]:
# === MERGE idempotente a la OBT ===
sql_merge = f"""
MERGE INTO {SCHEMA_AN}.{OBT_TABLE} AS tgt
USING {SCHEMA_AN}.{OBT_TABLE}_STG AS stg
ON tgt.trip_hash = stg.trip_hash

WHEN MATCHED AND (
     NVL(tgt.total_amount, -1)      <> NVL(stg.total_amount, -1)
  OR NVL(tgt.tip_amount, -1)        <> NVL(stg.tip_amount, -1)
  OR NVL(tgt.payment_type, -1)      <> NVL(stg.payment_type, -1)
  OR NVL(tgt.rate_code_id, -1)      <> NVL(stg.rate_code_id, -1)
  OR NVL(tgt.trip_duration_min, -1) <> NVL(stg.trip_duration_min, -1)
  OR NVL(tgt.avg_speed_mph, -1)     <> NVL(stg.avg_speed_mph, -1)
  OR NVL(tgt.tip_pct, -1)           <> NVL(stg.tip_pct, -1)
) THEN UPDATE SET
  pickup_datetime       = stg.pickup_datetime,
  dropoff_datetime      = stg.dropoff_datetime,
  pickup_date           = stg.pickup_date,
  pickup_hour           = stg.pickup_hour,
  dropoff_date          = stg.dropoff_date,
  dropoff_hour          = stg.dropoff_hour,
  day_of_week           = stg.day_of_week,
  month                 = stg.month,
  year                  = stg.year,

  pu_location_id        = stg.pu_location_id,
  pu_zone               = stg.pu_zone,
  pu_borough            = stg.pu_borough,
  do_location_id        = stg.do_location_id,
  do_zone               = stg.do_zone,
  do_borough            = stg.do_borough,

  service_type          = stg.service_type,
  vendor_id             = stg.vendor_id,
  vendor_name           = stg.vendor_name,
  rate_code_id          = stg.rate_code_id,
  rate_code_desc        = stg.rate_code_desc,
  payment_type          = stg.payment_type,
  payment_type_desc     = stg.payment_type_desc,
  trip_type             = stg.trip_type,
  trip_type_desc        = stg.trip_type_desc,

  passenger_count       = stg.passenger_count,
  trip_distance         = stg.trip_distance,
  store_and_fwd_flag    = stg.store_and_fwd_flag,

  fare_amount           = stg.fare_amount,
  extra                 = stg.extra,
  mta_tax               = stg.mta_tax,
  tip_amount            = stg.tip_amount,
  tolls_amount          = stg.tolls_amount,
  improvement_surcharge = stg.improvement_surcharge,
  congestion_surcharge  = stg.congestion_surcharge,
  airport_fee           = stg.airport_fee,
  total_amount          = stg.total_amount,
  e_hail_fee            = stg.e_hail_fee,

  trip_duration_min     = stg.trip_duration_min,
  avg_speed_mph         = stg.avg_speed_mph,
  tip_pct               = stg.tip_pct,

  run_id                = stg.run_id,
  ingested_at_utc       = stg.ingested_at_utc,
  source_service        = stg.source_service,
  source_year           = stg.source_year,
  source_month          = stg.source_month,
  source_path           = stg.source_path,

  nb03_run_id           = stg.nb03_run_id,
  nb03_processed_at_utc = stg.nb03_processed_at_utc

WHEN NOT MATCHED THEN INSERT (
  pickup_datetime, dropoff_datetime,
  pickup_date, pickup_hour, dropoff_date, dropoff_hour, day_of_week, month, year,
  pu_location_id, pu_zone, pu_borough, do_location_id, do_zone, do_borough,
  service_type, vendor_id, vendor_name, rate_code_id, rate_code_desc,
  payment_type, payment_type_desc, trip_type, trip_type_desc,
  passenger_count, trip_distance, store_and_fwd_flag,
  fare_amount, extra, mta_tax, tip_amount, tolls_amount,
  improvement_surcharge, congestion_surcharge, airport_fee, total_amount, e_hail_fee,
  trip_duration_min, avg_speed_mph, tip_pct,
  run_id, ingested_at_utc, source_service, source_year, source_month, source_path,
  nb03_run_id, nb03_processed_at_utc, trip_hash
) VALUES (
  stg.pickup_datetime, stg.dropoff_datetime,
  stg.pickup_date, stg.pickup_hour, stg.dropoff_date, stg.dropoff_hour, stg.day_of_week, stg.month, stg.year,
  stg.pu_location_id, stg.pu_zone, stg.pu_borough, stg.do_location_id, stg.do_zone, stg.do_borough,
  stg.service_type, stg.vendor_id, stg.vendor_name, stg.rate_code_id, stg.rate_code_desc,
  stg.payment_type, stg.payment_type_desc, stg.trip_type, stg.trip_type_desc,
  stg.passenger_count, stg.trip_distance, stg.store_and_fwd_flag,
  stg.fare_amount, stg.extra, stg.mta_tax, stg.tip_amount, stg.tolls_amount,
  stg.improvement_surcharge, stg.congestion_surcharge, stg.airport_fee, stg.total_amount, stg.e_hail_fee,
  stg.trip_duration_min, stg.avg_speed_mph, stg.tip_pct,
  stg.run_id, stg.ingested_at_utc, stg.source_service, stg.source_year, stg.source_month, stg.source_path,
  stg.nb03_run_id, stg.nb03_processed_at_utc, stg.trip_hash
);
"""
run_sql(sql_merge, schema=SCHEMA_AN)
print('✓ MERGE OBT ejecutado (idempotente).')


✓ MERGE OBT ejecutado (idempotente).


## 3) Conteo y auditoría

In [39]:
rows = run_sql(f"SELECT COUNT(*) FROM SPARK_DATA.analytics.OBT_TRIPS")
total = rows[0][0] if rows else 0
print('Filas en OBT:', total)

run_sql(f"""
INSERT INTO {SCHEMA_AN}.LOAD_AUDIT(load_ts_utc, run_id, step, rows_affected, details)
VALUES(CURRENT_TIMESTAMP(), '{os.getenv("RUN_ID","nb03_build_obt")}', 'NB03_BUILD_{OBT_TABLE}', {total}, '{SCHEMA_AN}.{OBT_TABLE}')
""")
print('✓ Auditoría registrada')


Filas en OBT: 818414540
✓ Auditoría registrada


## 4) Sanity checks básicos

In [40]:
checks = {
  'coherencia_fechas': f"SELECT COUNT(*) FROM {SCHEMA_AN}.{OBT_TABLE} WHERE dropoff_datetime < pickup_datetime",
  'duracion_negativa': f"SELECT COUNT(*) FROM {SCHEMA_AN}.{OBT_TABLE} WHERE trip_duration_min < 0",
  'dist_negativa':     f"SELECT COUNT(*) FROM {SCHEMA_AN}.{OBT_TABLE} WHERE trip_distance < 0",
  'monto_negativo':    f"SELECT COUNT(*) FROM {SCHEMA_AN}.{OBT_TABLE} WHERE total_amount < 0",
}
for name,q in checks.items():
    out = run_sql(q)
    print(name, '→', out[0][0] if out else None)

coherencia_fechas → 0
duracion_negativa → 0
dist_negativa → 0
monto_negativo → 2018051


In [44]:
## 5) Reingesta de un mes para probar idempotencia

In [41]:
# Parámetros de demo (ajusta según tu dataset)
# Reingesta de un mes para probar idempotencia.

TEST_YEAR  = int(os.getenv('TEST_YEAR', 2019))
TEST_MONTH = int(os.getenv('TEST_MONTH', 5))
OBT_TABLE= os.getenv('AN_OBT_TABLE', 'OBT_TRIPS')

# Volvemos a construir el STG pero limitando a un mes
sql_stg_subset = f"""
CREATE OR REPLACE TEMP TABLE {SCHEMA_AN}.{OBT_TABLE}_STG AS
SELECT * FROM {SCHEMA_AN}.{OBT_TABLE}_STG
WHERE year = {TEST_YEAR} AND month = {TEST_MONTH};
"""
run_sql(sql_stg_subset, schema=SCHEMA_AN)

before = run_sql(f"SELECT COUNT(*) FROM {SCHEMA_AN}.{OBT_TABLE} WHERE year={TEST_YEAR} AND month={TEST_MONTH}")[0][0]

# Re-ejecuta el mismo MERGE (no debería aumentar el conteo)
run_sql(sql_merge, schema=SCHEMA_AN)

after  = run_sql(f"SELECT COUNT(*) FROM {SCHEMA_AN}.{OBT_TABLE} WHERE year={TEST_YEAR} AND month={TEST_MONTH}")[0][0]
print(f"Idempotencia {TEST_YEAR}-{TEST_MONTH:02d}: antes={before:,} / después={after:,} (esperado: iguales)")


Idempotencia 2019-05: antes=8,129,493 / después=8,129,493 (esperado: iguales)
